In [ ]:
import numpy as np
from torch.utils.data import DataLoader, Dataset, random_split
import torchvision.transforms as transforms
import matplotlib.pyplot as plt


In [ ]:

data_path = "C:\\Users\\ACER\\PycharmProjects\\pythonProject17\\data\\"
images_orig = np.load(data_path + "Sunflower_Stages.npy", allow_pickle=True)
labels_orig = np.load(data_path + "Sunflower_Stages_Labels.npy", allow_pickle=True)


In [ ]:
num_classes = len(np.unique(labels_orig))
example_images = {}
for img, label in zip(images_orig, labels_orig):
    label = int(label)
    if label not in example_images:
        example_images[label] = img
    if len(example_images) == num_classes:
        break

# Побудова графіка
plt.figure(figsize=(16, 4))
for i in range(num_classes):
    img = example_images[i]
    img = img[..., ::-1]
    plt.subplot(1, num_classes, i + 1)
    plt.imshow(img)
    plt.title(f"Стадія {i + 1}")
    plt.axis('off')
plt.suptitle("Приклади зображень до трансформацій для кожного класу", fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
images_orig = images_orig.astype(np.float32) / 255.0
labels_orig = labels_orig.reshape(-1)

print(f"Форма зображень: {images_orig.shape}")
print(f"Форма міток: {labels_orig.shape}")

In [ ]:
images = images_orig
labels = labels_orig

In [ ]:

class SunflowerDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = torch.tensor(self.labels[idx], dtype=torch.long)

        if isinstance(image, np.ndarray):
            image = torch.from_numpy(image).permute(2, 0, 1)

        if self.transform:
            image = self.transform(image)

        return image, label


train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=15),
    transforms.RandomResizedCrop(size=224, scale=(0.8, 1.0)),
    transforms.RandomPerspective(distortion_scale=0.5, p=0.5),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],   # mean
                         [0.229, 0.224, 0.225])   # std
])

val_test_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],   # mean
                         [0.229, 0.224, 0.225])   # std
])

total_size = len(images)
train_size = int(0.8 * total_size)
val_size = int(0.1 * total_size)
test_size = total_size - train_size - val_size

indices = list(range(total_size))
train_indices, val_indices, test_indices = random_split(indices, [train_size, val_size, test_size])

train_data = SunflowerDataset([images[i] for i in train_indices], [labels[i] for i in train_indices], transform=train_transform)
val_data = SunflowerDataset([images[i] for i in val_indices], [labels[i] for i in val_indices], transform=val_test_transform)
test_data = SunflowerDataset([images[i] for i in test_indices], [labels[i] for i in test_indices], transform=val_test_transform)

batch_size = 32

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, pin_memory=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, pin_memory=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, pin_memory=True)

print(f"Train size: {len(train_data)}, Validation size: {len(val_data)}, Test size: {len(test_data)}")


In [ ]:
def show_images(loader, num_images=5):
    data_iter = iter(loader)
    images, labels = next(data_iter)

    fig, axes = plt.subplots(1, num_images, figsize=(15, 5))

    for i in range(num_images):
        img = images[i].permute(1, 2, 0).numpy()
        img = np.clip(img, 0, 1)
        img = img[..., ::-1]

        label = labels[i].item()

        axes[i].imshow(img)
        axes[i].set_title(f"Stage: {label+1}")
        axes[i].axis("off")

    plt.show()

show_images(val_loader)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torch.utils.data import DataLoader
import time


In [ ]:
model = models.resnet18(pretrained=True)

for param in model.parameters():
    param.requires_grad = False

model.fc = nn.Linear(model.fc.in_features, num_classes)

print(model)


In [ ]:
mobilenet = models.mobilenet_v2(pretrained=True)

for param in mobilenet.parameters():
    param.requires_grad = False

mobilenet.classifier[1] = nn.Linear(mobilenet.classifier[1].in_features, num_classes)

for param in mobilenet.classifier[1].parameters():
    param.requires_grad = True

print(mobilenet)


In [ ]:
from efficientnet_pytorch import EfficientNet

efficientnet = EfficientNet.from_pretrained('efficientnet-b0')

for param in efficientnet.parameters():
    param.requires_grad = False

efficientnet._fc = nn.Linear(efficientnet._fc.in_features, num_classes)

for param in efficientnet._fc.parameters():
    param.requires_grad = True

print(efficientnet)


In [ ]:
criterion = nn.CrossEntropyLoss()

# optimizer = optim.Adam(model.fc.parameters(), lr=0.001)
optimizer = optim.Adam(mobilenet.classifier[1].parameters(), lr=1e-4)
# optimizer = optim.Adam(efficientnet._fc.parameters(), lr=1e-4)


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

precision_list = []
recall_list = []
f1_list = []
inference_times_ms = []

num_epochs = 25


train_losses = []
train_accuracies = []
val_accuracies = []
patience = 5
best_val_acc = 0
epochs_no_improve = 0

start_time = time.time()

for epoch in range(num_epochs):
    mobilenet.train()
    running_loss = 0.0
    correct, total = 0, 0

    for images, labels in train_loader:

        optimizer.zero_grad()
        outputs = mobilenet(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_acc = 100 * correct / total
    avg_train_loss = running_loss / len(train_loader)

    train_losses.append(avg_train_loss)
    train_accuracies.append(train_acc)

    # --- ОЦІНКА НА ВАЛІДАЦІЇ ---
    mobilenet.eval()
    val_correct, val_total = 0, 0
    val_preds = []
    val_true = []
    inference_time_epoch = 0

    with torch.no_grad():
        for images, labels in val_loader:

            start_infer = time.time()
            outputs = mobilenet(images)
            _, predicted = torch.max(outputs, 1)
            end_infer = time.time()

            inference_time_epoch += (end_infer - start_infer)

            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

            val_preds.extend(predicted.cpu().numpy())
            val_true.extend(labels.cpu().numpy())

    val_acc = 100 * val_correct / val_total
    val_accuracies.append(val_acc)

    # --- ДОДАТКОВІ МЕТРИКИ ---
    precision = precision_score(val_true, val_preds, average='macro')
    recall = recall_score(val_true, val_preds, average='macro')
    f1 = f1_score(val_true, val_preds, average='macro')
    avg_infer_ms = inference_time_epoch * 1000 / len(val_loader.dataset)

    precision_list.append(precision)
    recall_list.append(recall)
    f1_list.append(f1)
    inference_times_ms.append(avg_infer_ms)

    print(f"Epoch [{epoch+1}/{num_epochs}] - "
          f"Train Loss: {avg_train_loss:.4f}, "
          f"Train Acc: {train_acc:.2f}%, "
          f"Val Acc: {val_acc:.2f}%, "
          f"Precision: {precision:.2f}, Recall: {recall:.2f}, F1: {f1:.2f}, "
          f"Infer time: {avg_infer_ms:.2f} ms")

end_time = time.time()
print(f"\nТренування завершено за {end_time - start_time:.2f} секунд")


In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 4, figsize=(18, 9))
axes = axes.flatten()

axes[0].plot(range(1, len(train_losses)+1), train_losses, label='Train Loss', color='blue')
axes[0].set_title("train/loss")
axes[0].grid(True)

axes[1].plot(range(1, len(train_accuracies)+1), train_accuracies, label='Train Accuracy', color='green')
axes[1].plot(range(1, len(val_accuracies)+1), val_accuracies, label='Val Accuracy', color='orange')
axes[1].set_title("metrics/accuracy")
axes[1].legend()
axes[1].grid(True)

axes[2].plot(range(1, len(precision_list)+1), precision_list, label='Precision', color='darkgreen')
axes[2].set_title("metrics/precision")
axes[2].grid(True)

axes[3].plot(range(1, len(recall_list)+1), recall_list, label='Recall', color='darkred')
axes[3].set_title("metrics/recall")
axes[3].grid(True)

axes[4].plot(range(1, len(f1_list)+1), f1_list, label='F1 Score', color='purple')
axes[4].set_title("metrics/F1")
axes[4].grid(True)

axes[5].plot(range(1, len(inference_times_ms)+1), inference_times_ms, label='Inference time', color='teal')
axes[5].set_title("metrics/inference_time (ms)")
axes[5].grid(True)

axes[6].axis("off")
axes[7].axis("off")

plt.suptitle("Криві метрик для EfficientNet-B0", fontsize=16)
plt.tight_layout()
plt.show()


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import precision_recall_curve, average_precision_score
from sklearn.preprocessing import label_binarize
import matplotlib.pyplot as plt
import numpy as np

y_true = []
y_pred = []
y_score = []
class_names = ["Stage 1", "Stage 2", "Stage 3", "Stage 4", "Stage 5", "Stage 6", "Stage 7", "Stage 8"]

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        y_true.extend(labels.numpy())
        y_pred.extend(preds.cpu().numpy())
        y_score.extend(torch.softmax(outputs, dim=1).cpu().numpy())


# Confusion matrix
cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
disp.plot(cmap=plt.cm.Blues)
plt.xticks(rotation=90)
plt.title('Confusion Matrix')
plt.show()

In [ ]:
y_true = label_binarize(y_true, classes=list(range(num_classes)))  # One-hot
y_score = np.array(y_score)

In [ ]:
plt.figure(figsize=(10, 8))
colors = plt.cm.get_cmap("tab10", num_classes)

average_precisions = []

for i in range(num_classes):
    precision, recall, _ = precision_recall_curve(y_true[:, i], y_score[:, i])
    ap = average_precision_score(y_true[:, i], y_score[:, i])
    average_precisions.append(ap)
    plt.plot(recall, precision, label=f"Stage {i+1} (AP={ap:.2f})", color=colors(i))

mean_ap = np.mean(average_precisions)
plt.plot([0, 1], [mean_ap, mean_ap], "k--", label=f"Mean AP={mean_ap:.3f}")

plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall curve (per class)")
plt.legend(loc="lower left")
plt.grid(True)
plt.show()


In [ ]:
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:

            outputs = model(images)
            _, predicted = torch.max(outputs, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Точність моделі на тестових даних: {accuracy:.2f}%")

evaluate_model(model, test_loader)


In [ ]:
from sklearn.metrics import classification_report

def classification_metrics(model, test_loader, class_names):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in test_loader:

            outputs = model(images)
            _, predicted = torch.max(outputs, 1)

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    print(classification_report(all_labels, all_preds, target_names=class_names))


classification_metrics(model, test_loader, class_names)


In [ ]:
torch.save(model.state_dict(), "sunflower_growth_model_resnet.pth")

# print("Модель збережено!")

In [ ]:
import torch
from torchvision import models
num_classes = 8

mobilenet = models.mobilenet_v2(pretrained=False)
mobilenet.classifier[1] = torch.nn.Linear(mobilenet.classifier[1].in_features, num_classes)
mobilenet.load_state_dict(torch.load("models/sunflower_growth_model_mobilenet3.pth", map_location="cpu"))
mobilenet.eval()


In [ ]:
imagenet_mean = np.array([0.485, 0.456, 0.406])
imagenet_std = np.array([0.229, 0.224, 0.225])

def denormalize(img_tensor):
    img = img_tensor.numpy().transpose((1, 2, 0))  # CHW -> HWC
    img = (img * imagenet_std) + imagenet_mean
    img = np.clip(img, 0, 1)
    return img


In [ ]:
import matplotlib.pyplot as plt
import torch
import numpy as np
class_names = ["Stage 1", "Stage 2", "Stage 3", "Stage 4", "Stage 5", "Stage 6", "Stage 7", "Stage 8"]

def show_class_examples(model, dataloader, class_names):
    model.eval()
    shown_classes = set()
    images_to_show = []
    true_labels = []
    predicted_labels = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for img, true_label, pred_label in zip(inputs, labels, preds.cpu()):
                if true_label.item() not in shown_classes:
                    shown_classes.add(true_label.item())
                    images_to_show.append(img.cpu())
                    true_labels.append(true_label.item())
                    predicted_labels.append(pred_label.item())
                if len(shown_classes) == len(class_names):
                    break
            if len(shown_classes) == len(class_names):
                break

    plt.figure(figsize=(16, 8))
    for idx, img in enumerate(images_to_show):
        plt.subplot(2, len(images_to_show)//2 + 1, idx + 1)
        img = denormalize(img)
        img = img[:, :, ::-1]
        plt.imshow(img)
        plt.title(f"Справжній: {class_names[true_labels[idx]]}\nПередбачено: {class_names[predicted_labels[idx]]}")
        plt.axis("off")

    plt.tight_layout()
    plt.show()


In [ ]:
show_class_examples(mobilenet, test_loader, class_names)
